# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

In [58]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_squared_error

In [59]:
RANDOM_STATE = 42

In [60]:
# Функция о пропусках в датасете

def missing(df):
    report = df.isna().sum().to_frame()
    report = report.rename(columns = {0: 'missing_values'})
    report['% of total'] = (report['missing_values'] / df.shape[0]).round(2)*100
    report.sort_values(by = 'missing_values', ascending = False)
    print('Количество пропущенных значений в датафрейме:')
    display(report)

In [61]:
# Функция для основной информации о датасете

def df_info(df):
    display(df.head())
    print('-' * 50)
    display(df.info())
    print('-' * 50)
    print()
    print('Количество дубликатов в датасете: ', data0.duplicated().sum())
    print()
    print('-' * 50)
    print()
    missing(df)

## Загрузка и подготовка данных

In [62]:
data0 = pd.read_csv('/datasets/geo_data_0.csv')
data1 = pd.read_csv('/datasets/geo_data_1.csv')
data2 = pd.read_csv('/datasets/geo_data_2.csv')

In [63]:
data0 = data0.set_index('id')
data1 = data1.set_index('id')
data2 = data2.set_index('id')

In [64]:
df_info(data0)

,f0,f1,f2,product
id,,,,
txEyH,0.705745,-0.497823,1.221170,105.280062
2acmU,1.334711,-0.340164,4.365080,73.037750
409Wp,1.022732,0.151990,1.419926,85.265647
iJLyR,-0.032172,0.139033,2.978566,168.620776
Xdl7t,1.988431,0.155413,4.751769,154.036647


--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
Index: 100000 entries, txEyH to 1CWhH
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   f0       100000 non-null  float64
 1   f1       100000 non-null  float64
 2   f2       100000 non-null  float64
 3   product  100000 non-null  float64
dtypes: float64(4)
memory usage: 3.8+ MB


None

--------------------------------------------------

Количество дубликатов в датасете:  0

--------------------------------------------------

Количество пропущенных значений в датафрейме:


,missing_values,% of total
f0,0,0.0
f1,0,0.0
f2,0,0.0
product,0,0.0


In [65]:
df_info(data1)

,f0,f1,f2,product
id,,,,
kBEdx,-15.001348,-8.276000,-0.005876,3.179103
62mP7,14.272088,-3.475083,0.999183,26.953261
vyE1P,6.263187,-5.948386,5.001160,134.766305
KcrkZ,-13.081196,-11.506057,4.999415,137.945408
AHL4O,12.702195,-8.147433,5.004363,134.766305


--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
Index: 100000 entries, kBEdx to relB0
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   f0       100000 non-null  float64
 1   f1       100000 non-null  float64
 2   f2       100000 non-null  float64
 3   product  100000 non-null  float64
dtypes: float64(4)
memory usage: 3.8+ MB


None

--------------------------------------------------

Количество дубликатов в датасете:  0

--------------------------------------------------

Количество пропущенных значений в датафрейме:


,missing_values,% of total
f0,0,0.0
f1,0,0.0
f2,0,0.0
product,0,0.0


In [66]:
df_info(data2)

,f0,f1,f2,product
id,,,,
fwXo0,-1.146987,0.963328,-0.828965,27.758673
WJtFt,0.262778,0.269839,-2.530187,56.069697
ovLUW,0.194587,0.289035,-5.586433,62.871910
q6cA6,2.236060,-0.553760,0.930038,114.572842
WPMUX,-0.515993,1.716266,5.899011,149.600746


--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
Index: 100000 entries, fwXo0 to V9kWn
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   f0       100000 non-null  float64
 1   f1       100000 non-null  float64
 2   f2       100000 non-null  float64
 3   product  100000 non-null  float64
dtypes: float64(4)
memory usage: 3.8+ MB


None

--------------------------------------------------

Количество дубликатов в датасете:  0

--------------------------------------------------

Количество пропущенных значений в датафрейме:


,missing_values,% of total
f0,0,0.0
f1,0,0.0
f2,0,0.0
product,0,0.0


**Промежуточный вывод:**

На данном этапе были импортированны библиотеки, разработаны функции (чтобы избежать повтора кода) для нахождения общей информации про датасеты, а также вывод информации о дубликатах и пропусках. Стоблеч *id* был переведен в индекс во всех df. Дубликатов и пропусков в df не обнаружено, типы данных также соответствуют логике.

## Обучение и проверка модели

In [67]:
def model(df, n_name, n):
    print(f'Предсказание модели для {n_name} региона.')
    print()
    
    # Создаём словарь для хранения переменных
    variables = {}

    # Определяем X и y
    variables[f'X_{n}'] = df.drop('product', axis=1)
    variables[f'y_{n}'] = df['product']
    
    # Разбиваем данные на обучающую и валидационную выборки
    variables[f'X_{n}_train'], variables[f'X_{n}_valid'], variables[f'y_{n}_train'], variables[f'y_{n}_valid'] = train_test_split(
        variables[f'X_{n}'],
        variables[f'y_{n}'],
        test_size=0.25,
        random_state=RANDOM_STATE
    )
    
    # Масштабируем данные
    scaler = StandardScaler()
    variables[f'X_{n}_train_scaled'] = scaler.fit_transform(variables[f'X_{n}_train'])
    variables[f'X_{n}_valid_scaled'] = scaler.transform(variables[f'X_{n}_valid'])
    
    # Обучаем модель
    model = LinearRegression()
    model.fit(variables[f'X_{n}_train_scaled'], variables[f'y_{n}_train'])
    
    # Делаем предсказания
    variables[f'predict_{n}'] = model.predict(variables[f'X_{n}_valid_scaled'])
    
    # Рассчитываем среднее и RMSE
    variables[f'mean_{n}'] = variables[f'predict_{n}'].mean()
    variables[f'rmse_{n}'] = mean_squared_error(variables[f'y_{n}_valid'], variables[f'predict_{n}'], squared=False)
       
    print(f"Средний запас предсказанного сырья: {round(variables[f'mean_{n}'], 2)} тыс. баррелей.")
    print(f"Значение метрики RMSE: {round(variables[f'rmse_{n}'], 2)}.")

    # Возвращаем словарь с результатами
    return variables


In [68]:
variables = model(data0, 'первого', 1)

for key, value in variables.items():
    globals()[key] = value 

Предсказание модели для первого региона.

Средний запас предсказанного сырья: 92.4 тыс. баррелей.
Значение метрики RMSE: 37.76.


D:\Anaconda\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [69]:
variables = model(data1, "второго" ,2)

for key, value in variables.items():
    globals()[key] = value 

Предсказание модели для второго региона.

Средний запас предсказанного сырья: 68.71 тыс. баррелей.
Значение метрики RMSE: 0.89.


D:\Anaconda\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [70]:
variables = model(data2, "третьего" , 3)

for key, value in variables.items():
    globals()[key] = value 

Предсказание модели для третьего региона.

Средний запас предсказанного сырья: 94.77 тыс. баррелей.
Значение метрики RMSE: 40.15.


D:\Anaconda\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


**Промежуточный вывод:**

В данном разделе были разделены данные на обучающую и валидационную выборки в соотношении 75:25 для обучения моделей для каждого региона. Результаты показали, что наибольший объем запасов сырья находится в третьей скважине, 94.8 тыс. баррелей. Однако у второй модели RMSE меньше всех (чем ближе значение к 1, тем точнее предсказание), при этом у двух других значения сильно завышены. 

## Подготовка к расчёту прибыли

In [71]:
#Количество скважин
n = 200

#Бюджет на разработку
budget = 10 * 10**9

#Доход с единицы продукта
revenue = 450000

#Сколько выделяется денег на одну скважину
budget_one = budget/n

#Сколько нужно единиц продукта, чтобы потрать весь бюджет
volume = budget_one / revenue

In [72]:
print(f'Расчет бюджета на одну скважину: {round(budget_one, 2)}.')
print(f'Количество единиц продукта для бюджета в одну скважину: {round(volume, 2)}.')

Расчет бюджета на одну скважину: 50000000.0.
Количество единиц продукта для бюджета в одну скважину: 111.11.


**Промежуточный вывод:**

На данном этапе было расчитано, сколько понадобится единиц сырья, чтобы потратить бюджет на одну скважину. Итого: Чтобы потратить бюджет на одну скважину в 50 миллионов рублей при доходе в 450 тысяч рублей с единицы продукта, нужно около 111 единиц продукта.

## Расчёт прибыли и рисков 

In [73]:
def profit(data, y_true, y_predict):
    
    # Добавляем столбцы с истинными и предсказанными значениями
    data['predict'] = y_predict
    data['true'] = y_true
    
    # Сортируем данные по предсказанным значениям в порядке убывания
    data = data.sort_values(by = 'predict', ascending = False)
    
    # Берем первые 200
    data = data.head(200)
    
    # Hfcxbnsdftv ghb,skm
    profite = data['true'].sum()*450000 - budget
    return profite

In [74]:
state = np.random.RandomState(12345)

In [75]:
prib0 = []
for i in range(1000):
    subsample = X0_valid.sample(n=500, replace = True, random_state=state)
    prib0.append(profit(subsample, y0_valid, pd.DataFrame(predict0, index = X0_valid.index)))

In [76]:
prib1 = []
for i in range(1000):
    subsample = X1_valid.sample(n=500, replace = True, random_state=state)
    prib1.append(profit(subsample, y1_valid, pd.DataFrame(predict1, index = X1_valid.index)))

In [77]:
prib2 = []
for i in range(1000):
    subsample = X2_valid.sample(n=500, replace = True, random_state=state)
    prib2.append(profit(subsample, y2_valid, pd.DataFrame(predict2, index = X2_valid.index)))

In [78]:
# Функция расчета средней прибыли и 95% доверительного интервала

def prib_score(prib, n):
    
    print(f'Средняя прибыль {n} региона {round(pd.Series(prib).mean(), 2)}.')

    lower = pd.Series(prib).quantile(0.025)
    upper = pd.Series(prib).quantile(0.975)
    print(f'95%-доверительный интервал {n} региона от {round(lower, 2)} до {round(upper, 2)}.')
    
    print(f'Доля отрицательных значений в нулевом регионе: {round(len([i for i in prib if i<0])/len(prib)*100, 2)}%.')

In [79]:
prib_score(prib0, 'первый')
print()
print("-" * 50)
print()
prib_score(prib1, 'второй')
print()
print("-" * 50)
print()
prib_score(prib2, 'третий')

Средняя прибыль первый региона 406278783.42.
95%-доверительный интервал первый региона от -117742136.49 до 911737050.75.
Доля отрицательных значений в нулевом регионе: 6.7%.

--------------------------------------------------

Средняя прибыль второй региона 441504277.59.
95%-доверительный интервал второй региона от 35728489.28 до 828006639.0.
Доля отрицательных значений в нулевом регионе: 1.6%.

--------------------------------------------------

Средняя прибыль третий региона 385213195.91.
95%-доверительный интервал третий региона от -164785166.11 до 888206234.2.
Доля отрицательных значений в нулевом регионе: 7.8%.


**Вывод:**

У второго региона вероятность убытка менее 2,5% (1,6%). В 95% доверительном интервале нет отрицательных значений, значит 95% значений - прибыль, а не убытки. Среднее значение прибыли в этом регионе - 441504277.6. Следовательно второй регион лучше всего подойдет для разработки